# Cubes with different element types
[![Google Collab Book](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/meyer-nils/torch-fem/blob/main/examples/basic/solid/cubes.ipynb)

All unit cubes are clamped at one end and subjected to a uniaxial strain.

In [1]:
import torch

from torchfem import Solid
from torchfem.elements import linear_to_quadratic
from torchfem.materials import AnisotropicDamage3D
from torchfem.mesh import cube_hexa, cube_tetra

# Set default data type to double precision
torch.set_default_dtype(torch.float64)


# A dummy orthotropic → isotropic-like material just for testing
E = 1000.0
nu = 0.3
G = E / (2 * (1 + nu))

material = AnisotropicDamage3D(
    E1=E, E2=E, E3=E,
    G12=G, G13=G, G23=G,
    nu12=nu, nu13=nu, nu23=nu,
    rho=1.0,
    Xt=1.0, Xc=1.0, Yt=1.0, Yc=1.0,
    S12=1.0, S13=1.0, S23=1.0,
    G_ft=1.0, G_fc=1.0, G_mt=1.0, G_mc=1.0,
    lc=1.0,
)


## 3D Cube with linear hexahedrons 


In [4]:
# Generate cube
nodes, elements = cube_hexa(5, 5, 5)

# Create model
box = Solid(nodes, elements, material)

# Assign boundary conditions
box.constraints[nodes[:, 0] == 0.0, :] = True
box.constraints[nodes[:, 0] == 1.0, 0] = True
box.displacements[nodes[:, 0] == 1.0, 0] = 0.1


# Solve
u, f, σ, F, α = box.solve()

# Plot
box.plot(u=u, node_property={"Disp": u})

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [6]:
print("u shape:", u.shape)
print("max disp:", u.abs().max())


u shape: torch.Size([125, 3])
max disp: tensor(0.1000)
